## Select a MSQ Tide file

In [ ]:
using CSV
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk

################################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# Pick directory containing MSQ .csv files
msq_directory = pick_folder()

# build list of all msq files in selected directory
msq_files = filter(x->occursin("_10min",x), readdir(msq_directory));
msq_files = msq_files[findall(x->endswith(uppercase(x), ".CSV"), msq_files)]

# Check whether any msq files exist in selected directory. If not, EXIT!
if length(msq_files) == 0
    println("No msq files found in "*msq_directory)
    exit;
else
    println(string(length(msq_files)) * " MSQ tide files found")
end

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, msq_files)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global file_choice = get_value(lb);
    
    # Select a msq file
    global infil = msq_directory * "\\" * file_choice[1]
    
    global title = file_choice[1]
    
    println("Selected ",infil)

    global msq_df = DataFrame(CSV.File(infil; header=false, skipto=41, delim=", ", ignorerepeated=true));
    msq_df.Date = DateTime.(Date.(strip.(msq_df.Column1),"dd/mm/yyyy"),Time.(strip.(msq_df.Column2),"HH:MM"))      
    rename!(msq_df,[:Column3,:Column4] .=> [:Ind, :Reading]);
    select!(msq_df, [:Date, :Ind, :Reading], Not([:Column1, :Column2]));
    msq_df.Reading = parse.(Float64,msq_df.Reading)
    
    msq_df.time_diff = [missing; diff(msq_df.Date)]
    
    global first_date = first(msq_df.Date)
    global last_date = last(msq_df.Date)
    
    year_diff = (Year(last_date) |> Dates.value) - (Year(first_date) |> Dates.value)
    
    # set X-axis scale based on amount of data
    if (year_diff == 0)
        mth_val = 1
    elseif (year_diff == 1)
        mth_val = 2
    elseif (year_diff == 2)
        mth_val = 3
    else
        mth_val = 6
    end
    
    # display plots to screen
    tm_tick = range(first_date,last_date,step=Month(mth_val))
    ticks = Dates.format.(tm_tick,"YY-mm")
    
    tides_plot = plot(msq_df.Date,msq_df.Reading, size = (1400, 600), label="",
##        marker = :circle, markersize = 1,
        xlims=(first_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,ytickfontsize=8, 
        framestyle = :box, legend=:bottomleft, title=title, 
        margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    display(tides_plot)
    
    start_date = first_date
    println(title,"\n")
    println("Mth Year  Gaps     Good     Minimum     Maximum     Mean      Range    St Devn")

    total_gaps = 0
    total_good = 0

    yearly_mean = 0
    no_of_months = 0
    
    while start_date <= last_date
        
        # select a month of data and store month of data in temporary df
        end_date = start_date + Dates.Month(1)
        month_df = msq_df[findall(start_date .<= msq_df.Date .< end_date),:]

        # determine total number of 10-minute records possible for the month
        total_possible = trunc(Int,(end_date - start_date) / Millisecond(1) * (1 / 600000))
        total_actual = nrow(month_df)
        
        # determine the number of gaps
        gaps = total_possible - total_actual

        # get totals of gaps and good data
        total_gaps = total_gaps + gaps
        total_good = total_good + total_actual

        # get monthly values
        month = Month(start_date) |> Dates.value
        year = Year(start_date) |> Dates.value
        monthly_min = minimum(month_df.Reading)
        monthly_max = maximum(month_df.Reading)
        monthly_mean = mean(month_df.Reading)
        monthly_std = std(month_df.Reading)
        monthly_range = monthly_max - monthly_min

        @printf("%3i %4i %5i %8i %10.3f %11.3f %10.3f %9.3f %9.3f\n", month, year, gaps, total_actual, monthly_min, monthly_max, monthly_mean, monthly_range, monthly_std)
        
        # add monthly mean to yearly total, and increment number of months used
        yearly_mean = yearly_mean + monthly_mean
        no_of_months = no_of_months + 1

        # print annual averages for current year of data
        if (Year(end_date) |> Dates.value) != (Year(start_date) |> Dates.value)
            println("=============================================================================")
            @printf("%s %4i %s %5.3f\n","                         Yearly average for",(Year(start_date) |> Dates.value),"= ", mean(msq_df[findall(Year.(msq_df.Date) .|> Dates.value .== year),:].Reading))
            @printf("%s%5.3f%s\n","                           From Monthly Averages = (",yearly_mean/no_of_months,")")
            println("=============================================================================\n")
            
            # reset counters
            yearly_mean = 0
            no_of_months = 0
        end

        # move to start of next month
        start_date = end_date

    end

    println("Total number of Gaps = ",total_gaps,"; Total number of Good values = ", total_good,"\n")
    
end

In [ ]:
# Check whether file is empty
if (stat(infil).size > 0) 

    fp = open(infil)

    header_line1 = readline(fp)
    header_line2 = readline(fp)

    identification = readline(fp)       
    site_number = rsplit(readline(fp), ",")[2]
    site_name = rsplit(readline(fp), ",")[2]
    Geo_quad = rsplit(readline(fp), ",")[2]
    latitude = rsplit(readline(fp), ",")[2]
    longiitude = rsplit(readline(fp), ",")[2]
    tidal_heights = rsplit(readline(fp), ",")[2]
    time_zone = rsplit(readline(fp), ",")[2]
    owner = rsplit(readline(fp), ",")[2]
    datum = rsplit(readline(fp), ",")[2]
    bench_mark = rsplit(readline(fp), ",")[2]
    bm_elevation = rsplit(readline(fp), ",")[2]

    readings = readline(fp)
    data_type = rsplit(readline(fp), ",")[2]
    data_class = rsplit(readline(fp), ",")[2]
    interval = rsplit(readline(fp), ",")[2]
    readings_reference = readline(fp)
    difference_from_datum = readline(fp)
    number_of_days = rsplit(readline(fp), ",")[2]
    number_of_valid_readings = rsplit(readline(fp), ",")[2]
    first_date = rsplit(readline(fp), ",")[2]
    first_ht = rsplit(readline(fp), ",")[2]
    last_date = rsplit(readline(fp), ",")[2]
    last_ht = rsplit(readline(fp), ",")[2]
    max_date = rsplit(readline(fp), ",")[2]
    max_ht = rsplit(readline(fp), ",")[2]
    min_date = rsplit(readline(fp), ",")[2]
    min_ht = rsplit(readline(fp), ",")[2]

    constituents = readline(fp)
    name_of_constituents_file = rsplit(readline(fp), ",")[2]
    owner_of_constituents_file = rsplit(readline(fp), ",")[2]
    model = rsplit(readline(fp), ",")[2]

    indicators = readline(fp)
    hw = rsplit(readline(fp), ",")[2]
    slack = rsplit(readline(fp), ",")[2]
    lw = rsplit(readline(fp), ",")[2]
    interval = rsplit(readline(fp), ",")[2]

    close(fp)

else

    println("Alert - " * infil * " is an empty file!")

end